In [1]:
import xmltodict
from typing import OrderedDict, List
from collections import OrderedDict
import abc


In [2]:
class BPMNParser():
    
    def simple_load(self, file_name:str)->OrderedDict:
        with open(file_name,"r") as file:
            process = xmltodict.parse(file.read()).get("bpmn:definitions", {}).get("bpmn:process", {})
        return process
          
    def load(self,file_name:str)->OrderedDict:
        process = self.simple_load(file_name)
        needed_checks = [self._check_start, self._check_end, self._check_task, self._check_exclusiveGateways, self._check_inclusiveGateways, self._check_parallelGateways]
        for check in needed_checks:
            process = check(process)
        return process
    
    def _transform_outgoing(self, outgoing, process):
        if type(outgoing) == str:
            flow = self.find_flow(process,outgoing)
            return {"@id": outgoing, "@name":flow.get("@name", "no_name"),"@targetRef": flow.get("@targetRef", None)}
        else:
            for key, flow_id in enumerate(outgoing):
                flow = self.find_flow(process, flow_id)
                outgoing[key] = {"@id": flow_id, "@name":flow.get("@name", "no_name"),"@targetRef": flow.get("@targetRef", None)}  
        return outgoing
    
    def _check_start(self, process:OrderedDict)->OrderedDict:
        start = process.get("bpmn:startEvent", None)
        assert type(start) == OrderedDict, "No Start Event found or multiple startevents"
        assert start.get("bpmn:incoming", None) is None, "startEvent can't have incoming flows"
        outgoing = start.get("bpmn:outgoing", None)
        assert type(outgoing) == str, "Start Event has mulitple Outgoing events or isn't connected"
        process["bpmn:startEvent"]["bpmn:outgoing"] = self._transform_outgoing(outgoing, process)
        return process
    
    def _check_end(self, process):
        end = process.get("bpmn:endEvent", None)
        assert end is not None, "Process needs an EndEvent!"
        if type(end) == OrderedDict:end = [end]
        for el in end:
            assert (el.get("bpmn:outgoing", None)) is None, "EndEvents cant have outgoing flows"
            inc = el.get("bpmn:incoming", None)
            assert type(inc) == str, "EndEvents cant have multiple incoming flows please use a closing Gate"
        return process
    
    def _check_task(self, process:OrderedDict)->OrderedDict:
        task = process.get("bpmn:task", None)
        if task is not None and type(task) == OrderedDict:
            assert type(task.get("@name", None)) is not None, "Task needs Name/Operation to be used in any meaningful way lol"
            assert type(task.get("bpmn:incoming", None)) == str, "Tasks needs exactly one incoming flow"
            out = task.get("bpmn:outgoing", None) 
            assert type(out) == str, "Task needs exactly one outgoing flow"
            process["bpmn:task"][i]["bpmn:outgoing"] = self._transform_outgoing(out,process)
        elif type(task) != list:return process
        else:
            for i,t in enumerate(task):
                assert type(t.get("@name", None)) is not None, "Task needs Name/Operation to be used in any meaningful way lol"
                assert type(t.get("bpmn:incoming", None)) == str, "Tasks needs exactly one incoming flow"
                out = t.get("bpmn:outgoing", None) 
                assert type(out) == str, "Task needs exactly one outgoing flow"
                process["bpmn:task"][i]["bpmn:outgoing"] = self._transform_outgoing(out,process)
        return process
    
    def _check_Gateways(self, process:OrderedDict, gate_type:str, default_op:str, check:bool)->OrderedDict:
        gateways = process.get(gate_type, None)
        if gateways is None: return process
        elif type(gateways) == list:
            for key,gateway in enumerate(gateways):
                inc = gateway.get("bpmn:incoming", None)
                out = gateway.get("bpmn:outgoing", None)
                assert (inc is not None or out is not None) and type(inc) != type(out), "gateways should be conncted properly"
                if type(inc) == list:
                    assert type(out) == str, "Closing Gate can't be also an Opening Gate"
                    process[gate_type][key]["@opening"] = False 
                    process[gate_type][key]["bpmn:outgoing"] = self._transform_outgoing(out, process)
                    #set default operation 
                    if gateway.get("@name", None) is None:
                        process[gate_type][key]["@name"] = default_op
                else:
                    assert type(inc) == str, "Opening Gate can't be also a Closing Gate"
                    process[gate_type][key]["@opening"] = True
                    process[gate_type][key]["bpmn:outgoing"] = self._transform_outgoing(out, process)
                    if check:
                        default = gateway.get("@default", None)
                        for o in out:
                            assert o.get("@name") != "no_name" or o.get("@id") == default, "Opening Gate needs all outgoing flows to have an Condition or be a default Flow"
                
        else:
            #determine if opening or closing Gate and see if check if all needed infos are present
            inc = gateways.get("bpmn:incoming", None)
            out = gateways.get("bpmn:outgoing", None)
            assert (inc is not None or out is not None) and type(inc) != type(out), "gateways should be conncted properly"
            if type(inc) == list:
                assert type(out) == str, "Closing Gate can't be also an Opening Gate"
                process[gate_type]["@opening"] = False 
                process[gate_type]["bpmn:outgoing"] = self._transform_outgoing(out, process)
                #set default operation 
                if gateways.get("@name", None) is None:
                    process[gate_type]["@name"] = default_op
            else:
                assert type(inc) == str, "Opening Gate can't be also a Closing Gate"
                process[gate_type]["@opening"] = True
                process[gate_type]["bpmn:outgoing"] = self._transform_outgoing(out, process)
                
                if check:
                    default = gateways.get("@default", None)
                    for o in out:
                        assert o.get("@name") != "no_name" or o.get("@id") == default, "Opening Gate needs all outgoing flows to have an Condition or be a default Flow"
                        
        return process
    
    def _check_exclusiveGateways(self, process):
        return self._check_Gateways(process, "bpmn:exclusiveGateway", "passtrough", True)

    def _check_inclusiveGateways(self, process):
        return self._check_Gateways(process, "bpmn:inclusiveGateway", "concat", True)

    def _check_parallelGateways(self, process):
        return self._check_Gateways(process, "bpmn:parallelGateway", "join", False)
    
    
    
    def find_flow(self,process, flow_id):
        for flow in process.get("bpmn:sequenceFlow"):
            if flow.get("@id") == flow_id:
                return flow
        return None
    def find_element(self, process, element_id)->dict:
        for key in process:
            if not key.startswith("@") and key != "bpmn:sequenceFlow":
                if type(process[key]) == list:
                    for count,item in enumerate(process[key]):
                        if process[key][count]["@id"] == element_id:
                            return {"type":key,
                                    "information":process[key][count]}
                else:
                    if process[key]["@id"] == element_id:
                            return {"type":key,
                                    "information":process[key]}
        return None
                                
            
parser = BPMNParser()
test = parser.load("all_elemts.bpmn")

In [ ]:
from uuid import uuid4
class Token():
    
    def __init__(self, context:str):
        self.data = None
        self.context:str = context
        self.taken_paths = 1
        self.id = str(uuid4())
    def add_context(self,element:str)->None:
        self.context = f"{self.context}-->{element}"
    def __repr__(self):
        return f"Token: {self.id}, Path taken: {self.context}"
    
    def __deepcopy__(self, memo):
        copied = Token(self.context)
        #here copy dataframe
        return copied

In [ ]:
class BPMNComponent:
    
    def __init__(self,process_definition:OrderedDict):
        self.id = process_definition.get("@id")
        self.name = process_definition.get("@name", "no_name")
        self.incoming = process_definition.get("bpmn:incoming", None)
        self.outgoing = process_definition.get("bpmn:outgoing", None)

    @abc.abstractclassmethod
    def execute(self):
        pass

In [ ]:
class StartEvent(BPMNComponent):
       
    def execute(self):
        token = Token(f"StartEvent :{self.id}{'('+self.name+')' if self.name != 'no_name' else ''}") 
        target = self.outgoing
        print(token)
        assert target["@targetRef"] != None, "missing refernce!!"
        return {
        "operation":"add",
        "elements" : [{"id":target["@targetRef"], "token":token}]
        }
st = StartEvent(test.get("bpmn:startEvent"))
st.execute()

In [ ]:
class EndEvent(BPMNComponent):
    
    def __init__(self,process_definition:OrderedDict, token:Token):
        self.token = token
        super().__init__(process_definition)
       
    def execute(self):
        self.token.add_context(f"EndEvent:{self.id}{'('+self.name+')' if self.name != 'no_name' else ''}")
        print(self.token)
        return {
        "operation":"end"
        }

In [ ]:
class Task(BPMNComponent):
    
    def __init__(self,process_definition:OrderedDict, token:Token):
        self.token = token
        super().__init__(process_definition)
    
    def execute(self):
        self.token.add_context(f"Task:{self.id}{'('+self.name+')' if self.name != 'no_name' else ''}")
        print(self.token)
        # here perform transformation action based on task name
        target = self.outgoing
        assert target["@targetRef"] != None, "missing refernce!!"
        return {
        "operation":"add",
        "elements" : [{"id":target["@targetRef"], "token":self.token}]
        }
    def __lt__(self, other):
        #here it needs to be definied which task should be done first lol
        return True

In [ ]:
class ExclusiveGateway(BPMNComponent):
    def __init__(self, process_definition:OrderedDict, token:Token):
        self.token = [token]
        self.opening = process_definition.get("@opening", False)
        self.default = process_definition.get("@default", None)
        super().__init__(process_definition)
    
    def execute(self):
        for token in self.token:
            token.add_context(f"ExclusiveGateway:{self.id}{'('+self.name+')' if self.name != 'no_name' else ''}")
            print(token)
        if self.opening:
            for el in self.outgoing:
                if el.get("@id", None) == self.default:
                    return {
                            "operation":"add",
                            "elements" : [{"id":el["@targetRef"], "token":self.token[0]}]
                            }
        else:
            #here you could customize the behaviour with some keywords
            new_token = self.token[0]
            return {
                    "operation":"add",
                    "elements" : [{"id":self.outgoing["@targetRef"], "token":new_token}]
                    }
        

In [ ]:
import copy 
class ParallelGateway(BPMNComponent):
    def __init__(self, process_definition:OrderedDict, token: Token):
        self.token = [token]
        self.opening = process_definition.get("@opening", False)
        super().__init__(process_definition)
        
    def execute(self):
        for token in self.token:
            token.add_context(f"ParallelGateway:{self.id}{'('+self.name+')' if self.name != 'no_name' else ''}")
            print(token)
        if self.opening:
            new_paths = len(self.outgoing)
            tba = [{"id":el["@targetRef"], "token":copy.deepcopy(self.token[0])} for el in self.outgoing]
            for key, obj in enumerate(tba):
                # print(key, obj)
                tba[key]["token"].taken_paths = new_paths
            return {"operation":"add", "elements":tba}
        else:
            token_len = len(self.token)
            if token_len == self.token[0].taken_paths:
                #do stuff
                new_token = self.token[0]
                new_token.taken_paths = 1
                return {
                        "operation":"add",
                        "elements" : [{"id":self.outgoing["@targetRef"], "token":new_token}]
                        }
            else:
                return {"operation":"repush"}
            
            

In [ ]:
import copy 
class InclusiveGateway(BPMNComponent):
    def __init__(self, process_definition:OrderedDict, token: Token):
        self.token = [token]
        self.opening = process_definition.get("@opening", False)
        self.default = process_definition.get("@default", None)
        super().__init__(process_definition)
        
    def execute(self):
        for token in self.token:
            token.add_context(f"InclusiveGateway:{self.id}{'('+self.name+')' if self.name != 'no_name' else ''}")
            print(token)
        if self.opening:
            tba = [{"id":el["@targetRef"], "token":copy.deepcopy(self.token[0])} for el in self.outgoing if el["@id"] == self.default]
            for key, obj in enumerate(tba):
                # print(key, obj)
                tba[key]["token"].taken_paths = len(tba)
            return {"operation":"add", "elements":tba}
        else:
            token_len = len(self.token)
            if token_len == self.token[0].taken_paths:
                #do stuff
                new_token = self.token[0]
                new_token.taken_paths = 1
                return {
                        "operation":"add",
                        "elements" : [{"id":self.outgoing["@targetRef"], "token":new_token}]
                        }
            else:
                return {"operation":"repush"}

In [ ]:
import heapq 

class BPMNEngine():
    def __init__(self,file_name:str):
        self.parser = BPMNParser()
        self.process = self.parser.load(file_name)
        self.elements = []
        self.find_start()
    
    def find_start(self):
        start = self.process.get("bpmn:startEvent")
        heapq.heappush(self.elements, (1,StartEvent(start)))
        
    def run(self):
        while self.elements:
            prio,cur = heapq.heappop(self.elements)
            print(prio, cur)
            next_step = cur.execute()
            if next_step["operation"] == "add":
                for element in next_step["elements"]:
                    self.add(element)
            elif next_step["operation"] == "repush":
                print(f"{cur} was repushed with prio: {prio+1}")
                heapq.heappush(self.elements, (prio+1, cur))
            elif next_step["operation"] == "end":
                if len(self.elements) == 0:
                    print("Process ended")
            else:
                pass
    def add(self, next_element):
        #check if element is already in queue
        for prio, el in self.elements:
            if el.id == next_element["id"]:
                el_type = type(el)
                if el_type == ParallelGateway or el_type == ExclusiveGateway or el_type == InclusiveGateway:
                    el.token.append(next_element["token"])
                return
        element = self.parser.find_element(self.process, next_element["id"])
        # print(element)
        if element["type"] == "bpmn:task":
            heapq.heappush(self.elements,(1,Task(element["information"], next_element["token"])))
        elif element["type"] == "bpmn:exclusiveGateway":
            heapq.heappush(self.elements, (5,ExclusiveGateway(element["information"], next_element["token"])))
        elif element["type"] == "bpmn:parallelGateway":
            heapq.heappush(self.elements, (5,ParallelGateway(element["information"], next_element["token"])))
        elif element["type"] == "bpmn:inclusiveGateway":
            heapq.heappush(self.elements, (5,InclusiveGateway(element["information"], next_element["token"])))
        elif element["type"] == "bpmn:endEvent":
            heapq.heappush(self.elements, (10,EndEvent(element["information"], next_element["token"])))
        else:
            pass
        
        
        
        
engine = BPMNEngine("all_elemts.bpmn")   
engine.run()

In [26]:
import re
s = "doNothing(1,2,3)"
r = re.match("(\w+)\([a-zA-Z0-9_:\[\]=, "".]*\)", s)
print(r)

<re.Match object; span=(0, 16), match='doNothing(1,2,3)'>
<re.Match object; span=(0, 16), match='doNothing(1,2,3)'>
